In [ ]:
'''
!!!!!!! Эта ячейка для запуска дедубликатора на Google Drive через Google Colab !!!!!!!

'''

from google.colab import drive
drive.mount('/content/drive')
# Устанавливаем рабочую директорию
pic_path = "/content/drive/My Drive/UII/Capture"
# Создаем директорию для дубликатов
dup_path = "/content/drive/My Drive/UII/Capture/Duplicates"

In [1]:
'''
!!!!!!! Эта ячейка для запуска дедубликатора на локальной машине через Jupyter Notebook (или IDE) !!!!!!!

'''

# Устанавливаем рабочую директорию
pic_path = r'C:\Users\anton\UII\AZavod_Ural'
# Создаем директорию для дубликатов
dup_path = r'C:\Users\anton\UII\AZavod_Ural\Duplicates'

In [40]:
'''
!!!!!!! Собственно сам дедубликатор !!!!!!!

'''

import os
import cv2
from PIL import Image
import numpy as np
from tqdm import tqdm
import shutil
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from concurrent.futures import ThreadPoolExecutor
import math
from collections import defaultdict

os.makedirs(dup_path, exist_ok=True)

def dedublicator(similar_threshold, show_dups, images_group):
    
    # Функция для динамического вычисления размера хэша изображения, в зависимости от размера изображения
    def dhash(image, base_size=64):
        initial_hash_size = max(image.shape[:2]) / base_size # Вычисляем начальный размер хэша
        hash_size = 2 ** math.ceil(math.log2(initial_hash_size)) # Округляем до ближайшей степени двойки
        hash_size = max(4, hash_size) # Ограничиваем минимальный размер хэша
        # Вычисляем хэш изображения
        resized = cv2.resize(image, (hash_size + 1, hash_size))
        diff = resized[:, 1:] > resized[:, :-1]
        return diff.flatten().astype(int), hash_size  # Возвращаем хэш в виде вектора битов и размерность хэш-матрицы

    # Функция для загрузки и хэширования изображений
    def load_and_hash_image(path):
        try:
            # Загружаем изображение с помощью Pillow
            with Image.open(path) as img:
                # Конвертируем изображение в формат BGR для OpenCV
                bgr_image = np.array(img.convert('RGB'))[:, :, ::-1]
            # Применяем функцию dhash
            dhash_value, _ = dhash(bgr_image)
            return (path, (dhash_value, bgr_image))
        except IOError:
            print(f"Не удалось загрузить изображение: {path}")
            return (path, (None, None))
        
    # Функция хэширования изображений в многопоточном режиме
    def process_images(paths):
        hash_dict = {}
        with ThreadPoolExecutor() as executor:
            results = list(executor.map(load_and_hash_image, paths))
        for path, (dhash_value, original_image) in results:
            if dhash_value is not None:
                hash_dict[path] = (dhash_value, original_image)
        return hash_dict

    # Создаем KD-дерево для быстрого поиска близких хэшей
    def create_kd_tree(hash_dict):
        hash_values = [val[0] for val in hash_dict.values()]
        tree = KDTree(hash_values)
        return tree

    # Функция для проверки схожести двух изображений
    def is_similar(hash1, hash2, similar_threshold):
        difference = np.sum(hash1 != hash2)
        similarity = (1 - difference / len(hash1)) * 100
        return similarity >= similar_threshold

    # Сортировка по группам и поиск дубликатов по KD-дереву
    def find_duplicates(hash_dict, kd_tree, similar_threshold):
        duplicate_groups = {}
        for file, (dhash_value, _) in hash_dict.items():
            similar_indices = kd_tree.query_ball_point(dhash_value, similar_threshold)
            duplicate_group = [file]
            for idx in similar_indices:
                similar_file = list(hash_dict.keys())[idx]
                if file != similar_file and is_similar(dhash_value, hash_dict[similar_file][0], similar_threshold):
                    duplicate_group.append(similar_file)
            if len(duplicate_group) > 1:
                duplicate_groups[len(duplicate_groups)] = list(set(duplicate_group))
        return duplicate_groups

    # Функция для поиска дубликатов между группами для их объединения по пересекающимся элементам
    def merge_duplicate_groups(duplicate_groups):
        merged_groups = []
        for group in duplicate_groups.values():
            # Ищем существующую группу, с которой есть пересечение
            found = False
            for merged in merged_groups:
                if set(group).intersection(merged):
                    merged.update(group)
                    found = True
                    break
            if not found:
                merged_groups.append(set(group))
        return merged_groups

    # Функция для отображения изображений
    def display_images(images):
        n_images = len(images)
        plt.figure(figsize=(5 * n_images, 5))  # Изменяем размер фигуры в зависимости от количества изображений
        for i, (title, image) in enumerate(images.items()):
            ax = plt.subplot(1, n_images, i + 1)  # Создаем подграфик для каждого изображения
            ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            ax.axis('off')
            ax.set_title(title)
        plt.show()

    # Основной код для обработки группы изображений
    hash_dict = process_images(images_group) # Вычисление хэшей
    if len(hash_dict) > 1:
        kd_tree = create_kd_tree(hash_dict) # Создание KD-дерева
        duplicate_groups = find_duplicates(hash_dict, kd_tree, similar_threshold) # Поиск дубликатов
        merged_duplicate_groups = merge_duplicate_groups(duplicate_groups) # Объединение групп дубликатов

        # Обработка и перемещение дубликатов
        for group in merged_duplicate_groups:
            if len(group) > 1: # Игнорируем группы с одним файлом
                original = sorted(group)[len(group) // 2] # Берем медианный файл в группе как оригинал
                duplicates = group - {original} # Остальные файлы считаем дубликатами
                # Выводим дубликаты и оригинал на экран
                if show_dups:
                    images_to_show = {f"Original": hash_dict[original][1]}
                    for i, dup in enumerate(duplicates):
                        images_to_show[f"Duplicate {i+1}"] = hash_dict[dup][1]
                    display_images(images_to_show)
                # Перемещаем похожие файлы
                for dup in duplicates:
                    dup_filename = os.path.basename(dup)
                    target_path = os.path.join(dup_path, dup_filename)
                    if not os.path.exists(target_path):
                        shutil.move(dup, target_path)

# Функция для группировки изображений по размерам
def group_images_by_size(directory):
    size_groups = defaultdict(list)
    for file in tqdm(os.listdir(directory), desc="Группировка по размеру", unit=" файлов"):
        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')):
            path = os.path.join(directory, file)
            try:
                image = Image.open(path)
                size = image.size
            except IOError:
                print(f"Failed to load image: {file}")
                continue
            if image is not None:
                size_groups[size].append(path)
    return size_groups.values()

#============================  Основное тело  ============================#

# Параметры

similar_threshold = 77 # степень похожести кадров (в %)
show_dups=False # выводить ли на экран оригинал с дубликатами

image_groups = group_images_by_size(pic_path)

# Ищем и убираем дубликаты в каждой подгруппе по размерам изображений
for group in tqdm(image_groups, desc="Обработка групп", unit=" групп"):
    dedublicator(similar_threshold, show_dups, group)

cnt_img_org = len(os.listdir(pic_path))  # Подсчет количества изображений в исходной папке
cnt_img_dup = len(os.listdir(dup_path)) # Подсчет количества перемещенных изображений
print(f"Похожие изображения успешно обработаны. Из {cnt_img_org+cnt_img_dup} файлов удалено {cnt_img_dup} похожих и дубликатов. Осталось {cnt_img_org} разных изображений")

Обработка групп: 100%|██████████| 446/446 [00:11<00:00, 40.10 групп/s] 

Похожие изображения успешно обработаны. Из 1361 файлов удалено 422 похожих и дубликатов. Осталось 939 разных изображений
